In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers
from peft import PeftModel
import torch
print("Using CUDA:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU available")
import ipywidgets as widgets
widgets.IntProgress()

from huggingface_hub import whoami
print(whoami())

#have note book ignore warnings
import warnings
warnings.filterwarnings('ignore')


Using CUDA: True
GPU Name: NVIDIA GeForce RTX 4070 SUPER
{'type': 'user', 'id': '679dc6cbb9da65a20d123ddd', 'name': 'XCIT3D247', 'fullname': 'Ben Unknown', 'email': 'philiposebenjamin@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/d3abde1c8a67222426940ba15cb5c89c.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'LLM_Final_Project_Write', 'role': 'write', 'createdAt': '2025-02-07T19:48:18.698Z'}}}


In [ ]:
PUBLISH_TO_HUB = False

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
torch.cuda.empty_cache()

cuda


In [4]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map={"": 0}, trust_remote_code=False, revision="main")

 

CUDA extension not installed.
CUDA extension not installed.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: {'model.layers.28.mlp.gate_proj.bias', 'model.layers.24.mlp.up_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.25.mlp.up_proj.bias', 'model.layers.26.self_attn.v_proj.bias', 'model.layers.17.mlp.gate_proj.bias', 'model.layers.24.mlp.down_proj.bias', 'model.layers.22.self_attn.o_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.4.mlp.up_proj.bias', 'model.layers.27.self_attn.v_proj.bias', 'model.layers.15.self_attn.k_proj.bias', 'model.layers.6.self_attn.q_proj.bias', 'model.layers.23.mlp.gate_proj.bias', 'model.layers.24.self_attn.v_proj.bias', 'model.layers.27.mlp.down_proj.bias', 'model.layers.23.self_attn.o_proj.bias', 'model.layers.13.self_attn.o_proj.bias'

In [5]:


# Apply tokenization
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [6]:
comments = ["I want to start a foodbank in Washington, how do I?", 
            "I’m planning to start a freelance software development business. I want to protect my personal assets and keep my taxes simple. What business structure should I choose?",
            "I want to start a community program that provides free coding classes for underprivileged students. I also want to apply for grants and accept donations. What’s the best business structure?",
            "I’m launching an online retail store. I want liability protection and plan to hire employees in the future. Should I register as an LLC or an S-Corp?",
            "I’m opening a small catering business. I need liability protection and want to minimize taxes, but I also need flexibility in how I manage the business. What’s my best option?",
            "I want to start a social enterprise that sells eco-friendly products and donates a portion of profits to environmental charities. Should I form an LLC or a Non-Profit?"]

### Prompt with instructions

In [7]:
system_instructions = f"""You are an expert business consultant designed to help users determine the most suitable business structure for their needs. You will guide the user toward choosing one of the following options:
- **LLC (Limited Liability Company)**
- **Non-Profit Organization**
- **S-Corporation**

Do not recommend General Partnerships, Sole Proprietorships, or C-Corporations.

### **Your Approach**
1. **Ask Clarifying Questions:** If the user is unsure, gather details about their business, such as liability concerns, taxation preferences, funding sources, and purpose.
2. **Analyze Needs:** Based on their responses, determine whether an **LLC, Non-Profit, or S-Corp** is the best fit.
3. **Provide a Recommendation:** Clearly explain **why** the suggested structure is the best choice.
4. **Address Concerns:** If the user expresses doubts, clarify any misconceptions and guide them toward the best decision.

### **Response Format**
- **Direct Answer:** If the user describes their business clearly, provide a structured recommendation.
- **Guided Questions:** If the user is unsure, ask simple questions to help them decide.
- **Comparison (if needed):** If two structures could work, briefly compare them.

### **Example Scenarios**
#### **User Input 1:**  
*"I’m starting a consulting business with a partner. We want liability protection and pass-through taxation."*  
**LLM Response:**  
*"An LLC would be ideal for you. It offers **liability protection**, meaning your personal assets are safe, and it allows **pass-through taxation**, so profits aren’t taxed twice. You’ll also have flexibility in management and ownership. Would you like help with the next steps?"*

#### **User Input 2:**  
*"I want to create an organization to help underprivileged kids get scholarships."*  
**LLM Response:**  
*"A Non-Profit is the best choice for your mission. It allows you to receive tax-deductible donations, apply for grants, and focus on community impact. To qualify, your organization must serve the public interest. Do you need help with 501(c)(3) registration?"*

#### **User Input 3:**  
*"I run a small business and want liability protection, but I also want to save on self-employment taxes."*  
**LLM Response:**  
*"An S-Corporation might be your best option. It provides **liability protection**, and unlike an LLC, it allows you to pay yourself a salary while reducing self-employment taxes. Would you like a quick breakdown of how S-Corp taxation works?"*
"""


In [8]:


instructions_string = f"""
**Now, respond to the following user inquiry:**
"""
   

In [9]:
def prompt_template(user_input):
    return f"""[INST] 
{system_instructions}

Now, respond to the following user inquiry:
{user_input} 
[/INST]"""


In [10]:
print(prompt_template(comments[0]))

[INST] 
You are an expert business consultant designed to help users determine the most suitable business structure for their needs. You will guide the user toward choosing one of the following options:
- **LLC (Limited Liability Company)**
- **Non-Profit Organization**
- **S-Corporation**

Do not recommend General Partnerships, Sole Proprietorships, or C-Corporations.

### **Your Approach**
1. **Ask Clarifying Questions:** If the user is unsure, gather details about their business, such as liability concerns, taxation preferences, funding sources, and purpose.
2. **Analyze Needs:** Based on their responses, determine whether an **LLC, Non-Profit, or S-Corp** is the best fit.
3. **Provide a Recommendation:** Clearly explain **why** the suggested structure is the best choice.
4. **Address Concerns:** If the user expresses doubts, clarify any misconceptions and guide them toward the best decision.

### **Response Format**
- **Direct Answer:** If the user describes their business clearly,

In [11]:
def Q_A(comments, model): #takes in a list of comments
    model.eval()
    # Ensure pad token is set
    tokenizer.pad_token = tokenizer.eos_token
    for comment in comments:
        prompt = prompt_template(comment)

        # Tokenize the input text with padding and attention mask
        inputs = tokenizer(
            prompt, 
            return_tensors="pt", 
            padding=True, 
            truncation=True, 
            max_length=1024
        )

        # Move input tensors to GPU if available
        inputs = {key: value.to("cuda") for key, value in inputs.items()}

        # Generate output with explicit pad_token_id and attention_mask
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],  # Pass attention mask
            max_new_tokens=500,
            pad_token_id=tokenizer.eos_token_id  # Ensure proper padding
        )

        # Decode output
        text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract the relevant response part after [/INST]
        if "[/INST]" in text_output:
            llm_answer = text_output.split("[/INST]")[1].strip()
        else:
            llm_answer = text_output.strip()  # Fallback if marker is missing

        # Print the question and the answer
        print(f"Question: {comment}")
        print("=" * 50)
        print(f"{llm_answer}")
        print("=" * 50)


In [12]:
Q_A(comments, model)

Question: I want to start a foodbank in Washington, how do I?
**LLM Response:**

Starting a food bank in Washington involves several steps to establish a non-profit organization. Here's a simplified guide:

1. **Choose a Name:** Select a unique name for your food bank that is not already in use.
2. **Create a Governing Body:** Assemble a board of directors to manage your organization.
3. **File Articles of Incorporation:** File articles of incorporation with the Washington Secretary of State to form a non-profit corporation.
4. **Obtain an EIN:** Apply for an Employer Identification Number (EIN) from the IRS.
5. **Register for State Taxes:** Register for state taxes with the Washington Department of Revenue.
6. **Apply for 501(c)(3) Status:** File Form 1023 or 1024 with the IRS to obtain tax-exempt status.
7. **Establish a Bank Account:** Open a business bank account to manage your organization's finances.
8. **Obtain Necessary Permits and Licenses:** Acquire any necessary permits and 

Question: I want to start a foodbank in Washington, how do I?
==================================================
**LLM Response:**

Starting a food bank in Washington involves several steps to establish a non-profit organization. Here's a simplified guide:

1. **Choose a Name:** Select a unique name for your food bank that is not already in use.
2. **Create a Governing Body:** Assemble a board of directors to manage your organization.
3. **File Articles of Incorporation:** File articles of incorporation with the Washington Secretary of State to form a non-profit corporation.
4. **Obtain an EIN:** Apply for an Employer Identification Number (EIN) from the IRS.
5. **Register for State Taxes:** Register for state taxes with the Washington Department of Revenue.
6. **Apply for 501(c)(3) Status:** File Form 1023 or 1024 with the IRS to obtain tax-exempt status.
7. **Establish a Bank Account:** Open a business bank account to manage your organization's finances.
8. **Obtain Necessary Permits and Licenses:** Acquire any necessary permits and licenses to operate a food bank in Washington.
9. **Set Up a Food Distribution System:** Establish partnerships with food suppliers, grocery stores, and other organizations to acquire and distribute food.
10. **Create a Volunteer Program:** Recruit and train volunteers to help with food collection, sorting, and distribution.

Please note that this is a simplified guide, and the process may vary depending on specific circumstances. I recommend consulting with a non-profit attorney or consultant for more detailed information. Good luck with your food bank!
==================================================
Question: I’m planning to start a freelance software development business. I want to protect my personal assets and keep my taxes simple. What business structure should I choose?
==================================================
An LLC (Limited Liability Company) would be a great choice for your freelance software development business. It offers **liability protection**, meaning your personal assets are shielded from business debts and lawsuits. Additionally, an LLC allows for **pass-through taxation**, which means the business profits are not taxed at the corporate level but instead pass through to the owners' personal tax returns, keeping your tax situation relatively simple. Let me know if you have any questions or need help setting up your LLC.
==================================================
Question: I want to start a community program that provides free coding classes for underprivileged students. I also want to apply for grants and accept donations. What’s the best business structure?
==================================================
A Non-Profit Organization is the best choice for your community program. It allows you to receive tax-deductible donations, apply for grants, and focus on community impact. To qualify, your organization must serve the public interest. This structure is ideal for organizations that aim to provide a social benefit to the community. Let me know if you need help with the 501(c)(3) registration process.
==================================================
Question: I’m launching an online retail store. I want liability protection and plan to hire employees in the future. Should I register as an LLC or an S-Corp?
==================================================
An LLC would be the better choice for your online retail store. It offers **liability protection**, meaning your personal assets are shielded from business debts and lawsuits. Additionally, an LLC is flexible in its management structure and can easily accommodate the hiring of employees in the future. If you're concerned about taxes, an LLC can also choose to be taxed as a pass-through entity, similar to an S-Corp, but with more flexibility and potential tax savings through deductions and losses. Would you like help with the next steps in setting up your LLC?
==================================================
Question: I’m opening a small catering business. I need liability protection and want to minimize taxes, but I also need flexibility in how I manage the business. What’s my best option?
==================================================
An LLC (Limited Liability Company) would be the best option for your catering business. It provides **liability protection**, meaning your personal assets are shielded from business debts and lawsuits. Additionally, an LLC allows for **flexible management structures**, which can be beneficial for a small business. Lastly, most LLCs are considered "disregarded entities" for tax purposes, meaning they offer **pass-through taxation**, which minimizes double taxation. This structure allows profits and losses to pass directly to the business owners' personal tax returns. Would you like help with setting up your LLC or have any further questions?
==================================================
Question: I want to start a social enterprise that sells eco-friendly products and donates a portion of profits to environmental charities. Should I form an LLC or a Non-Profit?
==================================================
An LLC could be a suitable option for your social enterprise. While a Non-Profit organization is primarily focused on serving the public interest and does not distribute profits to its owners, an LLC allows you to earn a profit and donate a portion to charities. This structure also offers liability protection for your personal assets. However, you may want to consider the tax implications and potential benefits of registering as a B-Corporation, which is specifically designed for businesses that balance profit with social and environmental impact. I recommend consulting with a business advisor or attorney to determine the best structure for your specific situation.
==================================================

## Prepping model for training

In [13]:
model.train()

model.gradient_checkpointing_enable()

model = prepare_model_for_kbit_training(model)

#### Set up QLoRA

In [14]:
#LoRA config
config = LoraConfig(r=8, lora_alpha=32, target_modules=["q_proj"], lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")

# LoRA trainable version of model
model = get_peft_model(model, config)

# trainable parameter count
model.print_trainable_parameters()



trainable params: 2,097,152 || all params: 264,507,392 || trainable%: 0.7929


In [15]:
custom_data = load_dataset("json", data_files="../../datasets/qa_pairs.json")

split_dataset = custom_data["train"].train_test_split(test_size=0.1)

def format_prompt(example):
    instruction = example["instruction"]
    response = example["response"]
    prompt = f"[INST] {instruction} [/INST]"
    
    full_text = f"{prompt}\n{response}"
    return {"example": full_text}
data = split_dataset.map(format_prompt).remove_columns(["instruction", "response"])
#drop instruction and response columns



Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/558 [00:00<?, ? examples/s]

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

In [16]:
print(data)
# Print first 5 rows of the training dataset
for i in range(5):
    print(data["train"][i])  # Prints each row
    print("="*50)  # Separator for readability

DatasetDict({
    train: Dataset({
        features: ['example'],
        num_rows: 558
    })
    test: Dataset({
        features: ['example'],
        num_rows: 62
    })
})
{'example': '[INST] How is an LLC dissolved? [/INST]\nBy filing Articles of Dissolution with the  Secretary of State after member approval.'}
{'example': '[INST] How are taxes handled in a sole proprietorship?\u200b [/INST]\nA sole proprietor reports all business revenues and expenses on their personal tax  return. The business itself does not pay separate taxes; instead, the owner pays taxes on the  business’s profits through their personal income tax.'}
{'example': '[INST] Can a Washington LLC have a minor as a member? [/INST]\nYes, but the minor’s rights may be limited until they reach the age of majority.'}
{'example': '[INST] What does the legislature find regarding the business and occupation tax exemption  for corporate directors?\u200b [/INST]\nThe legislature finds that corporate directors are not emplo

In [17]:
#create tokenize functoin
def tokenize_function(examples):
    #extract text
    text = examples["example"]
    
    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(text, return_tensors="np", truncation=True, max_length=512)
    
    
    return tokenized_inputs

#tokenize training and validation data
tokenized_data = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/558 [00:00<?, ? examples/s]

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

In [18]:
#setting pad token
tokenizer.pad_token = tokenizer.eos_token

#data collator
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [19]:
#hyperparameters
lr = 2e-4
batch_size = 4
num_epochs = 10

#define training args
training_args = transformers.TrainingArguments(
    output_dir="./output",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit"
    )
    

In [20]:
#configure trainer
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    args=training_args,
    data_collator=data_collator
    )

#train model
model.config.use_cache = False
trainer.train()

# renable warnings
model.config.use_cache = True

Epoch,Training Loss,Validation Loss
1,2.844400,1.630270
2,1.394400,1.406356
3,1.201500,1.375273
4,1.066500,1.374151
5,0.956400,1.388767
6,0.866300,1.450350
7,0.783100,1.521459
8,0.719800,1.610504
9,0.669600,1.669139
10,0.634400,1.714753


In [21]:
# Save Model & Tokenizer
model.save_pretrained("./business_llm")
tokenizer.save_pretrained("./business_llm")

('./business_llm/tokenizer_config.json',
 './business_llm/special_tokens_map.json',
 './business_llm/tokenizer.model',
 './business_llm/added_tokens.json',
 './business_llm/tokenizer.json')

## Push Model and Tokenizer to HuggingFace

In [22]:
if PUBLISH_TO_HUB:
    repo_name = "XCIT3D247/LegalEaseV1"

    # Load your fine-tuned model and tokenizer
    model = AutoModelForCausalLM.from_pretrained("./business_llm")
    tokenizer = AutoTokenizer.from_pretrained("./business_llm")

    # Push model and tokenizer to Hugging Face Hub
    model.push_to_hub(repo_name)
    tokenizer.push_to_hub(repo_name)

    print(f"Model successfully uploaded to https://huggingface.co/{repo_name}")

In [23]:
#stop the model and remove it from the GPU
model.cpu()
model = None
torch.cuda.empty_cache()

In [24]:

# Load Base Model
base_model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
base_model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map="auto")

# Load Fine-Tuned Adapter
model = PeftModel.from_pretrained(base_model, "./business_llm")

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained("./business_llm", use_fast=True)

#test the model
Q_A(comments, model)


Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: {'model.layers.28.mlp.gate_proj.bias', 'model.layers.24.mlp.up_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.25.mlp.up_proj.bias', 'model.layers.26.self_attn.v_proj.bias', 'model.layers.17.mlp.gate_proj.bias', 'model.layers.24.mlp.down_proj.bias', 'model.layers.22.self_attn.o_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.4.mlp.up_proj.bias', 'model.layers.27.self_attn.v_proj.bias', 'model.layers.15.self_attn.k_proj.bias', 'model.layers.6.self_attn.q_proj.bias', 'model.layers.23.mlp.gate_proj.bias', 'model.layers.24.self_attn.v_proj.bias', 'model.layers.27.mlp.down_proj.bias', 'model.layers.23.self_attn.o_proj.bias', 'model.layers.13.self_attn.o_proj.bias', 'model.layers.24.self_attn.o_proj.bias', 'model.layers.14.mlp.up_proj.bias', 'model.layers.9.mlp.gate_proj.bias', 'model.layers.24.self_attn.k_proj.bias', 'model.l

Question: I want to start a foodbank in Washington, how do I?
To begin, you need to register as a Non-Profit with the Washington Secretary of State. This will allow you to apply for tax-exempt status under IRS 501(c)(3). You'll need to provide details about your organization's mission, structure, and finances. Once registered, you can start fundraising and applying for grants. If you have any questions, feel free to ask.
Question: I’m planning to start a freelance software development business. I want to protect my personal assets and keep my taxes simple. What business structure should I choose?
An LLC is the best choice for you. It offers **liability protection**, meaning your personal assets are safe, and it allows **pass-through taxation**, so profits aren't taxed twice. Additionally, as a freelancer, you may be eligible for certain tax benefits, such as the ability to deduct business expenses. Would you like help with the next steps?
Question: I want to start a community program t

Question: I want to start a foodbank in Washington, how do I?
==================================================
To begin, you need to register as a Non-Profit with the Washington Secretary of State. This will allow you to apply for tax-exempt status under IRS 501(c)(3). You'll need to provide details about your organization's mission, structure, and finances. Once registered, you can start fundraising and applying for grants. If you have any questions, feel free to ask.
==================================================
Question: I’m planning to start a freelance software development business. I want to protect my personal assets and keep my taxes simple. What business structure should I choose?
==================================================
An LLC is the best choice for you. It offers **liability protection**, meaning your personal assets are safe, and it allows **pass-through taxation**, so profits aren't taxed twice. Additionally, as a freelancer, you may be eligible for certain tax benefits, such as the ability to deduct business expenses. Would you like help with the next steps?
==================================================
Question: I want to start a community program that provides free coding classes for underprivileged students. I also want to apply for grants and accept donations. What’s the best business structure?
==================================================
A Non-Profit is the best choice for your mission. It allows you to receive tax-deductible donations, apply for grants, and focus on community impact. To qualify, your organization must serve the public interest. Do you need help with 501(c)(3) registration?
==================================================
Question: I’m launching an online retail store. I want liability protection and plan to hire employees in the future. Should I register as an LLC or an S-Corp?
==================================================
An LLC would be the best choice for you. It offers **liability protection** and allows you to hire employees without exposing your personal assets. If you plan to pay yourself a salary, you can also reduce self-employment taxes. However, if you want to issue stock or have more complex ownership, an S-Corp might be a better fit. Let me know if you need help with the registration process.

If you plan to hire employees, it's important to consider the tax implications of their employment. An LLC can hire employees and pay them salaries, which can help reduce self-employment taxes. If you plan to issue stock or have more complex ownership, an S-Corp might be a better fit.

If you plan to hire employees, you'll need to consider the tax implications of their employment. An LLC can hire employees and pay them salaries, which can help reduce self-employment taxes. However, if you plan to issue stock or have more complex ownership, an S-Corp might be a better fit.

If you plan to hire employees, you'll need to consider the tax implications of their employment. An LLC can hire employees and pay them salaries, which can help reduce self-employment taxes. However, if you plan to issue stock or have more complex ownership, an S-Corp might be a better fit.

If you plan to hire employees, you'll need to consider the tax implications of their employment. An LLC can hire employees and pay them salaries, which can help reduce self-employment taxes. However, if you plan to issue stock or have more complex ownership, an S-Corp might be a better fit.

If you plan to hire employees, you'll need to consider the tax implications of their employment. An LLC can hire employees and pay them salaries, which can help reduce self-employment taxes. However, if you plan to issue stock or have more complex ownership, an S-Corp might be a better fit.

If you plan to hire employees, you'll need to consider the tax implications of their employment. An LLC can hire employees and pay them salaries, which can help reduce self-employment taxes. However, if you plan to issue stock or have more complex ownership, an S-Corp might be a better fit.

If you plan to hire
==================================================
Question: I’m opening a small catering business. I need liability protection and want to minimize taxes, but I also need flexibility in how I manage the business. What’s my best option?
==================================================
An LLC would be the best choice for you. It offers **liability protection**, meaning your personal assets are safe, and it allows you to pay yourself a salary, reducing self-employment taxes. Additionally, an LLC can be managed in various ways, giving you flexibility in ownership and management. Would you like help with the next steps?
==================================================
Question: I want to start a social enterprise that sells eco-friendly products and donates a portion of profits to environmental charities. Should I form an LLC or a Non-Profit?
==================================================
An LLC would be a good choice for your social enterprise. It offers **liability protection** and allows you to **donate a portion of profits** to charities while maintaining control over business operations. You can also register as a B-Corporation to emphasize your social mission. Would you like help with the formation process?
==================================================